In [17]:
import os
import numpy as np
import pandas as pd

ornament=" . "*3

root="d:/yuninze/nih.go.kr/koges"

workDirectory=os.path.join(root,"res")

kogesFeatherPath=os.path.join(workDirectory,"kogesSampleFull.feather")

indices=["id","date","visit"]
variableTypeMap={
    "연속형":"contiguous",
    "범주형":"sparse",
    "날짜":"datetime",
    "ID":"idx"
}

labelCols=["seq","visit","klass","variable","variableLabel","variableInfo","variableType","variableView"]

In [10]:
def _view(elem):
    if isinstance(elem,list):
        next={}
        for pair in tuple(q.split("=") for q in elem):
            if len(pair)==2:
                next[int(pair[0])]=pair[1]
        return next
    return elem

In [11]:
try:
    things={q.name[:-4]:pd.read_csv(q.path) for q in os.scandir(workDirectory) if q.name.lower().startswith("follow")}
    for thing in things:
        things[thing].columns=[q[4:] for q in things[thing].columns]
        things[thing]=things[thing].rename({"ID":"id","EDATE":"date"},axis=1).assign(visit=thing).set_index(indices)
    koges=pd.concat(things.values()).sort_index(level=(0,1))

except:
    print(ornament,"Attempting to read feather")
    if os.path.isfile(kogesFeatherPath):
        koges=pd.read_feather(kogesFeatherPath)
    else:
        raise OSError("")
    
else:
    print(ornament,"Got raw CSVs")

finally:
    print(ornament,"Saving to feather")
    koges.to_feather(os.path.join(kogesFeatherPath))

 .  .  .  Got raw CSVs
 .  .  .  Saving to feather


In [61]:
koges.iloc[11:21]

DATA_CLASS  SEX  AGE  EDU  MARRY  DRINK  \
id            date   visit                                                    
K_FOLLOW_0003 200505 FOLLOW_02_DATA        F05  NaN   54  NaN  66666      3   
              200806 FOLLOW_03_DATA        F05  NaN   57  NaN      2      3   
              201106 FOLLOW_04_DATA        F05  NaN   60  NaN      2      3   
              201406 FOLLOW_05_DATA        F05  NaN   63  NaN      2      3   
K_FOLLOW_0004 200504 FOLLOW_01_DATA        F05  2.0   60  2.0      2      1   
              200704 FOLLOW_02_DATA        F05  NaN   62  NaN  66666      1   
              201005 FOLLOW_03_DATA        F05  NaN   65  NaN      2      1   
              201304 FOLLOW_04_DATA        F05  NaN   68  NaN      3      1   
K_FOLLOW_0005 200402 FOLLOW_01_DATA        F19  1.0   49  3.0      2      3   
              200601 FOLLOW_02_DATA        F19  NaN   51  NaN      2      3   

                                        DRDU  TAKFQ    TAKAM  RICEFQ  ...  \
id            date   visit                                            ...   
K_FOLLOW_0003 200505 FOLLOW_02_DATA      NaN      0  77777.0       0  ...   
              200806 FOLLOW_03_DATA      NaN      0  77777.0       0  ...   
              201106 FOLLOW_04_DATA      NaN      0  77777.0       0  ...   
              201406 FOLLOW_05_DATA      NaN      1      2.0       0  ...   
K_FOLLOW_0004 200504 FOLLOW_01_DATA  77777.0      0  77777.0       0  ...   
              200704 FOLLOW_02_DATA      NaN      0  77777.0       0  ...   
              201005 FOLLOW_03_DATA      NaN      0  77777.0       0  ...   
              201304 FOLLOW_04_DATA      NaN      0  77777.0       0  ...   
K_FOLLOW_0005 200402 FOLLOW_01_DATA      4.0      0  77777.0       0  ...   
              200601 FOLLOW_02_DATA      NaN      0  77777.0       0  ...   

                                     DBP    HBA1C  GLU0  CREATININE  AST  ALT  \
id            date   visit                                                      
K_FOLLOW_0003 200505 FOLLOW_02_DATA   92  99999.0    90         1.2   33   38   
              200806 FOLLOW_03_DATA   92      5.0    81         1.1   27   32   
              201106 FOLLOW_04_DATA   89      5.7    92         1.3   24   24   
              201406 FOLLOW_05_DATA   86      5.5    92         1.1   30   27   
K_FOLLOW_0004 200504 FOLLOW_01_DATA   90      5.6    89         0.6   35   34   
              200704 FOLLOW_02_DATA   88  99999.0   106         0.7   20   17   
              201005 FOLLOW_03_DATA   86      6.1    95         0.7   33   36   
              201304 FOLLOW_04_DATA   84      5.4   106         0.7   32   29   
K_FOLLOW_0005 200402 FOLLOW_01_DATA   79      6.9    95         1.0   52   33   
              200601 FOLLOW_02_DATA   79  99999.0   124         1.1   25   12   

                                     TCHL  HDL   TG     INS0  
id            date   visit                                    
K_FOLLOW_0003 200505 FOLLOW_02_DATA   141   31   93     11.1  
              200806 FOLLOW_03_DATA   144   26  122     10.5  
              201106 FOLLOW_04_DATA   154   33  139     11.7  
              201406 FOLLOW_05_DATA   133   26  177      3.8  
K_FOLLOW_0004 200504 FOLLOW_01_DATA   182   47  123     12.6  
              200704 FOLLOW_02_DATA   194   53  109     18.8  
              201005 FOLLOW_03_DATA   169   34  111     14.2  
              201304 FOLLOW_04_DATA   177   34  137     15.7  
K_FOLLOW_0005 200402 FOLLOW_01_DATA   203   36  277      2.7  
              200601 FOLLOW_02_DATA   148   43  155  99999.0  

[10 rows x 62 columns]

In [35]:

labelExcel=[q.path for q in os.scandir(workDirectory) if q.name.lower().endswith(".xlsx")]
if len(labelExcel)!=0:
    label=pd.concat(
        [pd.read_excel(q.path,skiprows=2).set_axis(labelCols,axis=1) for q in labelExcel],
        axis=0
    ).set_index("table")
else:
    label=pd.read_feather(os.path.join(workDirectory,"dbs.feather"))

label.to_feather(os.path.join(workDirectory,"dbs.feather"))

In [49]:
labelView=label.drop("id",axis=1)
labelView.index=range(labelView.shape[0])
labelView=labelView.map(
    lambda q:"" if pd.isna(q) else q
)
labelView.loc[:,"variable"]=labelView.variable.map(
    lambda q:q[4:]
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q[q.find("\n")+1:]
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q[:q.rfind("\n")]
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    lambda q:q.replace("\n",", ") if isinstance(q,str) else q
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.apply(
    lambda q:q.split(", ") if q.__contains__(", ") else q
)
labelView.loc[:,"variableInfo"]=labelView.variableInfo.map(
    _view
)
labelView.loc[:,"variableType"]=list(map(
    lambda q:variableTypeMap[q],labelView.variableType
))

labelView.map(str).to_feather(os.path.join(workDirectory,"labelView.feather"))
labelView=pd.read_feather(os.path.join(workDirectory,"labelView.feather"))

In [52]:
labelView.sample(3)

,table,table_,variable,variableLabel,variableInfo,variableType,variableView
117,FOLLOW_04_DATA,생활습관,TAKAM,막걸리 음주 1회 주량,( ),contiguous,\n ▷ 막걸리를 한 번 드실 때의 평균총량은?\n ...
245,FOLLOW_01_DATA,생활습관,PSM,간접흡연 여부,"{1: '아니오', 2: ''}",sparse,\n■ 현재 다른 사람이 피우는 담배 연기를 맡는 경우(간접 흡연)가 있습니까? ...
100,FOLLOW_03_DATA,신체계측,DBP,이완기혈압,( ) mmH,contiguous,\n■ 혈압 sys ______...


In [53]:
print("Variable(s) missing in the book\n",
    [q for q in koges.columns if not q in labelView.variable.unique()]
)

Variable(s) missing in the book
 []


In [57]:
labelView.set_index()

ValueError: orient 'column' not understood

In [43]:
desc=koges.select_dtypes("number").query("variableType==contiguous").agg(["mean","median","std","max","min"]).T.round(3)

In [45]:
desc

,mean,median,std,max,min
SEX,1.558,2.0,0.497,2.0,1.0
AGE,58.764,58.0,9.671,86.0,40.0
EDU,702.283,2.0,8341.162,99999.0,1.0
MARRY,5272.043,2.0,18212.584,99999.0,1.0
DRINK,332.727,2.0,5742.123,99999.0,1.0
...,...,...,...,...,...
ALT,134.052,20.0,3318.188,99999.0,6.0
TCHL,301.888,190.0,3312.767,99999.0,89.0
HDL,155.919,44.0,3317.448,99999.0,19.0
TG,295.741,120.0,3921.632,99999.0,26.0


In [ ]:
sieve=(((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum,axis=1)).gt(18)

In [ ]:
koges.iloc[sieve.values,:].sample(5)

In [ ]:
(((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum)).describe()

In [ ]:
sieveCol=((koges.reset_index().set_index("T_ID").select_dtypes("number")).ge(60000)).apply(sum).ge(1600).values

In [ ]:
koges.loc[:,sieveCol]

In [ ]:
ukValueMapper={
    66666:np.nan,
    77777:None,
    99999:np.nan
}
kogesUkWeight=koges.map(lambda q:ukValueMapper[q] if q in (66666,77777,99999) else q).agg(lambda q:pd.notna(q).value_counts(normalize=True)).T*100

In [ ]:
kogesUkWeight[True].to_frame().describe()

In [ ]:
kogesUkWeight

In [ ]:
koges_=koges.map(lambda q:np.nan if q in (66666,77777,99999) else q).reset_index()

In [ ]:
x=koges_.T_ID
y=koges_.loc[:,[q for q in koges_.columns if q!="T_ID"]]

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


Eq=BayesianRidge(tol=tuple(1e-1*(q) for q in (np.logspace(1,2,10))))

Imputer=IterativeImputer(Eq)

Imputer.fit